<a href="https://colab.research.google.com/github/codedog1229/MachineLearning/blob/main/homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!nvidia-smi

Thu May  4 06:16:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#下载资料
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

In [4]:
#导包
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [5]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# **Dataset**

In [6]:
class COVID19Dataset(Dataset):
  '''
  X:Features.
  Y:Target,if none,do prediction
  '''
  def __init__(self,x,y=None):
    if y is None:
      self.y = y
    else:
      self.y = torch.FloatTensor(y)
    self.x = torch.FloatTensor(x)

  def __getitem__(self,idx):
    if self.y is None:
      return self.x[idx]
    else:
      return self.x[idx],self.y[idx]
  
  def __len__(self):
    return len(self.x)

# **Neural Network Model**
Try out different model architectures by modifying the class below.

In [24]:
class My_Model(nn.Module):
  def __init__(self,input_dim):
    super(My_Model,self).__init__()
    # TODO：修改模型的结构，注意维度。
    self.layers = nn.Sequential(
        nn.Linear(input_dim,16),
        nn.ReLU(),
        nn.Linear(16,8),
        nn.ReLU(),
        nn.Linear(8,1)
    )
  
  def forward(self,x):
    x = self.layers(x)
    x = x.squeeze(1)
    return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [9]:
def select_feat(train_data, valid_data, test_data, select_all=True):
  '''Select useful features to perform regression'''
  #最后一行为y值
  y_train,y_valid = train_data[:,-1],valid_data[:,-1]
  raw_x_train,raw_x_valid,raw_x_test = train_data[:,:-1],valid_data[:,:-1],test_data
  if(select_all):
    feat_idx = list(range(raw_x_train.shape[1]))
  else:
    feat_idx = [0,1,2,3,4] # TODO: Select suitable feature columns.
  return raw_x_train[:,feat_idx],raw_x_valid[:,feat_idx],raw_x_test[:,feat_idx],y_train,y_valid

# Training Loop

In [ ]:
# Configurations
#`config` contains hyper-parameters for training and the path to save your model.

In [13]:
def trainer(train_loader, valid_loader, model, config, device):
  #定义Loss函数
  criterion = nn.MSELoss(reduction='mean')
  #定义优化器
  optimizer = torch.optim.SGD(model.parameters(),lr=config['learning_rate'],momentum=0.9)
  #SummaryWriter 类是您记录数据以供 TensorBoard 使用和可视化的主要入口
  writer = SummaryWriter()

  if not os.path.isdir('./models'):
    os.mkdir('./models') #创建一个文件夹用来保存模型
  
  n_epochs = config['n_epochs']
  best_loss = math.inf
  step = 0
  early_stop_count = 0

  for epoch in range(n_epochs):
    model.train()#将模型设置为训练模式
    loss_record = []
    #tqdm 是一个可视化训练进度的软件包。
    train_pbar = tqdm(train_loader, position=0, leave=True)

    for x,y in train_pbar:
      optimizer.zero_grad()#设置梯度为0
      x,y = x.to(device),y.to(device)#将张量放进对应的设备
      pred = model(x)
      #计算loss
      loss = criterion(pred,y)
      #反向传播计算梯度
      loss.backward()
      #更新参数
      optimizer.step()
      step += 1
      loss_record.append(loss.detach().item())

      #展示当前的进度和loss
      train_pbar.set_description(f'Epoch[{epoch+1}/{n_epochs}]')
      train_pbar.set_postfix({'loss':loss.detach().item()})

    mean_train_loss = sum(loss_record)/len(loss_record)
    writer.add_scalar('Loss/train',mean_train_loss,step)

    model.eval()#将模型设置为验证模式
    loss.record = []
    for x,y in valid_loader:
      x,y = x.to(device),y.to(device)
      with torch.no_grad():
        pred = model(x)
        loss = criterion(pred,y)

      loss_record.append(loss.item())
    
    mean_valid_loss = sum(loss_record)/len(loss_record)
    print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
    writer.add_scalar('Loss/valid', mean_valid_loss, step)

    #判断
    if mean_valid_loss<best_loss:
      best_loss = mean_valid_loss
      torch.save(model.state_dict(),config['save_path']) #保存你最好的模型
      print('Saving model with loss{:.3f}...'.format(best_loss))
      early_stop_count = 0
    else: 
      early_stop_count += 1                                               
    #提早结束训练
    if early_stop_count >= config['early_stop']:
      print('\nModel is not improving, so we halt the training session.')
      return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 3000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-5,              
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [14]:
%cd '/content/drive/MyDrive/ML2023/homework1'

/content/drive/MyDrive/ML2023/homework1


In [18]:
# Set seed for reproducibility
same_seed(config['seed'])
# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])
# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

train_data size: (2408, 89) 
valid_data size: (601, 89) 
test_data size: (997, 88)


In [20]:
# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])
# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

number of features: 88


In [21]:
train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train),COVID19Dataset(x_valid, y_valid),COVID19Dataset(x_test)

In [22]:
# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

# Start Training!

In [25]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch[1/3000]: 100%|██████████| 10/10 [00:02<00:00,  3.56it/s, loss=290]


Epoch [1/3000]: Train loss: 334.8469, Valid loss: 280.7511
Saving model with loss280.751...


Epoch[2/3000]: 100%|██████████| 10/10 [00:00<00:00, 125.45it/s, loss=175]


Epoch [2/3000]: Train loss: 141.2223, Valid loss: 139.2577
Saving model with loss139.258...


Epoch[3/3000]: 100%|██████████| 10/10 [00:00<00:00, 135.15it/s, loss=176]


...

Epoch [420/3000]: Train loss: 159.3668, Valid loss: 158.2871


Epoch[421/3000]: 100%|██████████| 10/10 [00:00<00:00, 115.06it/s, loss=120]


Epoch [421/3000]: Train loss: 157.0874, Valid loss: 156.5578


Epoch[422/3000]: 100%|██████████| 10/10 [00:00<00:00, 140.59it/s, loss=164]


Epoch [422/3000]: Train loss: 159.4523, Valid loss: 160.0338

Model is not improving, so we halt the training session.


# Testing
The predictions of your model on testing set will be stored at pred.csv.

In [26]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

100%|██████████| 4/4 [00:00<00:00, 476.65it/s]
